### 善甲狼週報 V2.0 - 可觀測性分析平台 Colab 執行入口
點擊下方的按鈕選擇啟動模式：

In [ ]:
from IPython.display import display, HTML
import ipywidgets as widgets

button_normal_mode = widgets.Button(description='[ 正常模式 ]', button_style='success')
button_debug_mode = widgets.Button(description='[ 除錯模式 ]', button_style='warning')
output_area = widgets.Output()

def on_button_normal_clicked(b):
    with output_area:
        output_area.clear_output()
        print('您選擇了「正常模式」。開始執行應用程式...')
        # 在此處添加「正常模式」的執行腳本
        # !git clone <YOUR_REPO_URL>善甲狼週報V2
        # %cd 善甲狼週報V2
        # !pip install -r backend/requirements.txt
        # !npm --prefix frontend install
        # !npm --prefix frontend run build
        # !nohup uvicorn backend.app.main:app --host 0.0.0.0 --port 8000 > logs/backend.log 2>&1 &
        # !npm install -g serve
        # !nohup serve -s frontend/out -p 3000 > logs/frontend.log 2>&1 &
        print('（模擬）正常模式啟動完成。')

def on_button_debug_clicked(b):
    with output_area:
        output_area.clear_output()
        print('您選擇了「除錯模式」。開始執行健康度檢測...')
        # 在此處添加「除錯模式」的執行腳本
        # !python health_check_final.py
        print('（模擬）除錯模式執行完成。')

button_normal_mode.on_click(on_button_normal_clicked)
button_debug_mode.on_click(on_button_debug_clicked)

display(widgets.HBox([button_normal_mode, button_debug_mode]))
display(output_area)